In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2
import os

In [ ]:
DATADIR = '/gdrive/My Drive/Kidney Cancer'


CATEGORIES = ["Normal", "Tumor"]

In [ ]:
IMG_SIZE = 100

In [ ]:
all_data =[]

def create_all_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                all_data.append([new_array,class_num])
            except Exception as e:
                pass
create_all_data()


In [ ]:
print("Toplam Görüntü Sayısı: ", len(all_data))

Toplam Görüntü Sayısı:  10000


In [ ]:
X = []
y = []

for categories, label in all_data:
  X.append(categories)
  y.append(label)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [ ]:
# %70 %30 bloğu

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [ ]:

import tensorflow as tf

train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

model = Sequential()
model.add(Conv2D(128, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 128)       3584      
                                                                 
 batch_normalization (Batch  (None, 98, 98, 128)       512       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 128)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 49, 49, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 128)       147584    
                                                                 
 batch_normalization_1 (Bat  (None, 47, 47, 128)       5

In [ ]:
from tensorflow import keras

callback_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor = 'val_accuracy', save_best_only=True, verbose=3
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=3)
]

In [ ]:
history = model.fit(x_train, train_yCl,
                    batch_size=128,
                    validation_data = (x_val, valid_yCl),
                    callbacks = callback_list,
                    epochs = 20
                    )

Epoch 1/20
57/57 [==============================] - ETA: 0s - loss: 0.2248 - accuracy: 0.9251
Epoch 1: val_accuracy improved from -inf to 0.88875, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


57/57 [==============================] - 26s 253ms/step - loss: 0.2248 - accuracy: 0.9251 - val_loss: 0.3595 - val_accuracy: 0.8888
Epoch 2/20
57/57 [==============================] - ETA: 0s - loss: 0.0319 - accuracy: 0.9896
Epoch 2: val_accuracy improved from 0.88875 to 0.97375, saving model to model.h5
57/57 [==============================] - 13s 227ms/step - loss: 0.0319 - accuracy: 0.9896 - val_loss: 0.0847 - val_accuracy: 0.9737
Epoch 3/20
57/57 [==============================] - ETA: 0s - loss: 0.0208 - accuracy: 0.9925
Epoch 3: val_accuracy did not improve from 0.97375
57/57 [==============================] - 12s 216ms/step - loss: 0.0208 - accuracy: 0.9925 - val_loss: 0.1283 - val_accuracy: 0.9450
Epoch 4/20
57/57 [==============================] - ETA: 0s - loss: 0.0114 - accuracy: 0.9960
Epoch 4: val_accuracy did not improve from 0.97375
57/57 [==============================] - 12s 218ms/step - loss: 0.0114 - accuracy: 0.9960 - val_loss: 0.0879 - val_accuracy: 0.9525
Epoch 5

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
# SVM model
svm_model = SVC()

# Parametre ızgarası
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 0.1, 1],
    'degree': [2, 3, 4],
}
num_features = x_train.shape[1] * x_train.shape[2] * x_train.shape[3]
x_train = x_train.reshape(-1, num_features)
# Grid Search
grid_search_svm = GridSearchCV(estimator=svm_model, param_grid=param_grid, scoring='accuracy', cv=5,  error_score='raise')
grid_search_svm.fit(x_train, y_train)


# En iyi parametreleri ve en iyi skoru görüntüleme
print("En iyi parametreler: ", grid_search_svm.best_params_)
print("En iyi skor: ", grid_search_svm.best_score_)

In [ ]:
score_valid = model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = model.evaluate(x_test, test_yCl)
print("Validation Accuracy: ", score_test[1])

score_train = model.evaluate(x_train, train_yCl)
print("Validation Accuracy: ", score_train[1])

25/25 [==============================] - 21s 797ms/step - loss: 4.9970 - accuracy: 0.4988
Validation Accuracy:  0.4987500011920929
63/63 [==============================] - 49s 783ms/step - loss: 4.8790 - accuracy: 0.4975
Validation Accuracy:  0.4975000023841858
225/225 [==============================] - 179s 797ms/step - loss: 4.9776 - accuracy: 0.4942
Validation Accuracy:  0.49416667222976685


In [ ]:
import itertools
from sklearn.metrics import confusion_matrix, classification_report
#Plot the confusion matrix. Set Normalize = True/False
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Reds):

    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=2)
        cm[np.isnan(cm)] = 0.0
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def evaluate_model(model, x_test, y_test):
    # Accuracy Score
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test.reshape(-1, 1), axis=1)
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    print(f"Accuracy: {accuracy}")

    # Precision Score
    precision = precision_score(y_test_classes, y_pred_classes, average='weighted', zero_division = 0)
    print(f"Precision: {precision}")

    # Recall Score
    recall = recall_score(y_test_classes, y_pred_classes, average='weighted', zero_division = 0)
    print(f"Recall: {recall}")

    # F1 Score
    f1 = f1_score(y_test_classes, y_pred_classes, average='weighted', zero_division = 0)
    print(f"F1-Score: {f1}")

    # Classification Report
    print("Classification Report:")
    print(classification_report(y_test_classes, y_pred_classes))

    # Confusion Matrix
    Y_pred = model.predict(x_test)
    y_pred_model = np.argmax(Y_pred, axis=1)
    cm_model = confusion_matrix(y_test, y_pred_model)
    plot_confusion_matrix(cm_model, CATEGORIES, normalize=False, title='Karmaşıklık Matrisi (Confusion Matrix)')

    # Modeli değerlendir
    evaluate_model(model, x_test, y_test)